# Entity Resolution (Active Learning + Crowdsourcing)

## Step 1. Read Data

In [ ]:
import csv
data = []
with open('restaurant.csv', 'rb') as csvfile:
    reader = csv.reader(csvfile)
    header = reader.next()
    for row in reader:
        data.append(row)
print "Num of Rows:", len(data)
print "Header:", header
print "First Row:", data[0]
print "Second Row:", data[1]

## Step 2. Removing Obviously Non-duplicate Pairs

In [ ]:
from reprowd.utils.simjoin import *

def joinkey_func(row):
    # concatenate 'name', 'address', 'city' and 'type', and
    # tokensize the concatenated string into a set of words 
    return wordset(' '.join(row[1:])) 

key_row_list = [(joinkey_func(row) , row) for row in data]
sj = SimJoin(key_row_list)
result = sj.selfjoin(0.4)  
result.sort(key=lambda x: -x[2])
simpairs = [(row1, row2) for (key1, row1), (key2, row2), sim in result] 

print "Num of Pairs: ", len(data)*(len(data)-1)/2
print "Num of Similar Pairs: ", len(simpairs)
print "The Most Similar Pair: ", simpairs[0]

## Step 3. Active Learning

### Featurization

In [ ]:
from reprowd.utils.simjoin import *

# Use this function to transform each pair to a feature vector 
def featurize(pair):
    row1, row2 = pair
    
    #cleaning
    row1 = [alphnum(x.lower()) for x in row1]
    row2 = [alphnum(x.lower()) for x in row2] 
    
    # features
    f1 = editsim(row1[1], row2[1])
    f2 = jaccard_w(row1[1], row2[1])
    f3 = editsim(row1[2], row2[2])
    f4 = jaccard_w(row1[2], row2[2])
    f5 = editsim(row1[3], row2[3])
    f6 = editsim(row1[4], row2[4])
    
    return (f1, f2, f3, f4, f5, f6)

print "The feature vector of the first pair: ", featurize(simpairs[0])

### Initialization

In [ ]:
from reprowd.crowdcontext import *
from reprowd.presenter.text import TextCmp

# choose the most/least similar five pairs as initial training data
init_pairs = simpairs[:5] + simpairs[-5:]

def map_func(row_pair):
    row1, row2 = row_pair
    return {'obj1': zip(header, row1), 'obj2':zip(header, row2)}

cc = CrowdContext()

presenter = TextCmp().set_name("Finding Duplicate Resturants") \
                .set_short_name("dup-resturant") \
                .set_question("Are they the same restaurant?") 
        
        
crowddata = cc.CrowdData(init_pairs, "active-ER") \
                .set_presenter(presenter, map_func) \
                .publish_task().get_result().quality_control("mv")

matches = []
nonmatches = []
for pair, label in zip(crowddata.data['object'], crowddata.data['mv']):
    if label == 'Yes':
        matches.append(pair)
    else:
        nonmatches.append(pair)
        
print "Number of matches: ", len(matches)
print "Number of nonmatches: ", len(nonmatches)

### Interactive Model Training

<font color='red'>**Here is the only code you need to write in this assignment.**</font>  

In [ ]:
labeled_pairs = matches + nonmatches
unlabeled_pairs = [p for p in simpairs if p not in labeled_pairs]
batch_size = 2
iter_num = 5

#<-- Write Your Code -->


### Evaluation

In [ ]:
import json

with open('ground_truth.json') as f:    
    true_matches = json.load(f)
    

def evaluate(identified_matches, true_matches):
    n = 0
    for (x, y) in identified_matches:
        if [x, y] in true_matches or [y, x] in true_matches:
            n += 1
    precision = n*1.0/len(identified_matches)
    recall = n*1.0/len(true_matches)
    fscore = 2*precision*recall/(precision+recall)
    return (precision, recall, fscore) 
            

sp_features = np.array([featurize(sp) for sp in simpairs])
label = model.predict(sp_features)
pair_label = zip(simpairs, label)

identified_matches = []
for pair, label in pair_label:
    if label == 1:
        identified_matches.append(((pair[0][0], pair[1][0])))
        
precision, recall, fscore = evaluate(identified_matches, true_matches)

print "Precision:", precision
print "Recall:", recall
print "Fscore:", fscore
   